### Import Libraries

In [5]:
import pandas as pd
import numpy as np

# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

#Sci-kit libraries
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier


### Loading in Data

In [6]:
posts = pd.read_csv('../data/wine_beer_concatenated.csv')

In [7]:
posts.head()

,selftext,subreddit
0,Resources for a newbie home winemaker I want t...,winemaking
1,A question about kit wine I’ve made quite a fe...,winemaking
2,"Riesling - My First Wine Hi Everyone,\n\nI am ...",winemaking
3,Persimmon Wine Straining? Hello! This is my fi...,winemaking
4,Wire used for Trellis Hey all - I'm planting s...,winemaking


In [8]:
posts.tail()

,selftext,subreddit
4297,"Any input on this recipe 6 lb - Pale Malt, Mar...",Homebrewing
4298,"Bottling Tepache Hello brewers, \n\na couple d...",Homebrewing
4299,Wiring a spa panel for ebiab gfi I've been all...,Homebrewing
4300,What can I ferment at 60-65°F? The basement in...,Homebrewing
4301,Does a decoction mash with wheat leave a bread...,Homebrewing


In [9]:
posts['subreddit'].value_counts()

Homebrewing    2367
winemaking     1935
Name: subreddit, dtype: int64

### Binarizing subreddit column for targeting

In [10]:
posts['subreddit'] = posts['subreddit'].map({'winemaking': 0, 'Homebrewing': 1})

In [11]:
posts['subreddit'].value_counts()

1    2367
0    1935
Name: subreddit, dtype: int64

# Baseline Model
**Interpretation**
* If we guessed at random we would select Homebrewing subreddit 55% of the time and winemaking subreddit 45% of the time. 

In [12]:
posts['subreddit'].value_counts(normalize = True)

1    0.550209
0    0.449791
Name: subreddit, dtype: float64

### Create Target and Predictor Variables

In [13]:
X = posts['selftext']
y = posts['subreddit']

### Train/Test/Split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, 
                                                   random_state = 42, 
                                                   stratify = y)

In [15]:
## creating a list of our two chosen vectorizers to iterate through in our grid search
vectorizer = [CountVectorizer(), TfidfVectorizer()]

In [ ]:
## creating variables to accept tuning parameters
max_feat = [300, 500]  
ngram_range = [(1, 3), (1, 2)] 
stop_words = [None, 'english'] 
max_df = [0.9, 0.8] 

Create loop to loop through vectorizers, create a pipeline and loop through potential models while grid search looks for optimal hyperparameters. 

Information on setting voting classifier parameter to 'soft' or 'hard'. https://towardsdatascience.com/how-voting-classifiers-work-f1c8e41d30ff

Soft on even number of classifiers. Hard on odd number of classifiers. Since I am using 4 classifiers I tried to set it to soft but kept getting error messages. I am imagining one of the classifiers is not working well with 'soft.' I returned to using 'hard' and it worked fine. 

In [22]:
## creating any empty results list to capture our cv_results_ at the end of each iteration
results = []

## looping through both vectorizers
for vect in vectorizer:
    
    #### Pipeline for our Vectorizer and Classifier Models ####
    pipeline = Pipeline([
        ('vect', vect),
        ('clf', LogisticRegression())])
    
    instantiations = [ #### Beginning of Instantiations List ####
        {
            ### Log Reg Vect Hyperparameters ###
            
            'vect__max_features': max_feat,
            'vect__stop_words': stop_words,
            'vect__ngram_range': ngram_range,
            'vect__max_df': max_df,
            
            ## Instantiate Log Reg and Hyperparamaters
            'clf': (LogisticRegression(solver='liblinear'), ), ## setting our first classifier model
            'clf__penalty': ('l1', 'l2'),
            'clf__C': (.5, 1.0), 
        }, 
    #######
        {
            
            ### Multinomial Bayes Vect Hyperparameters ###
            'vect__max_features': max_feat,
            'vect__stop_words': stop_words,
            'vect__ngram_range': ngram_range,
            'vect__max_df': max_df,
            
            ## Instantiate Mulinomial NB and Hyperparamters ##
            'clf': (MultinomialNB(), ),  
            'clf__alpha': (.5, 1.0)
        },
    #######   
        {
            ### SVC Vect Hyperparameters ###
            'vect__max_features': max_feat,
            'vect__stop_words': stop_words,
            'vect__ngram_range': ngram_range,
            'vect__max_df': max_df,
            
            ## SVC Instantiation and Hyperparameters
            'clf': (SVC(gamma='scale', ), ),
            'clf__kernel': ('rbf', 'poly') 
        },
    #######  
        {
            ### RandomForestClassifier Vect Hyperparameters ###
            'vect__max_features': max_feat,
            'vect__stop_words': stop_words,
            'vect__ngram_range': ngram_range,
            'vect__max_df': max_df,
            
            ## Instantiate RandomForestClassifier ##
            'clf': (RandomForestClassifier(n_estimators=50, min_samples_split=5), ),
        },
    #######
        {
            ### Voting Classifier Vect Hyperparameters ###
            'vect__max_features': max_feat,
            'vect__stop_words': stop_words,
            'vect__ngram_range': ngram_range,
            'vect__max_df': max_df,
            
            ## Instantiating Ensemble Voting Classifier ##
            'clf': (VotingClassifier(estimators=[('lr', LogisticRegression()), 
                                                 ('rf', RandomForestClassifier()), 
                                                 ('mnb', MultinomialNB()), 
                                                 ('svc', SVC())],                                           
                                            voting='hard'), )
        }    
                    ] #### end of instantiations list ####
    
    
    #### Grid Search ####
    
    grid_search = GridSearchCV(pipeline, 
                               instantiations,
                               cv=3,
                               n_jobs=-1,
                               verbose=1,
                               return_train_score=True)
    
    #### Output Results ####
    
    ## running an if statement to print the type of vectorizer used
    if vect == vectorizer[0]:
        vect_string = "CountVectorizer"
    
    else:
        vect_string = "Tf-IDF Vectorizer"
    
    ## fitting our model and printing our best scores and parameters
    grid_search.fit(X_train, y_train)
    print(f'''Best score for {vect_string} is: 
    {round(grid_search.best_score_, 4)}
    ''')
    print(grid_search.best_params_)
    print("")
    
    ## appending our cv_results_ to the end of results
    results.append(grid_search.cv_results_)

Fitting 3 folds for each of 160 candidates, totalling 480 fits
Best score for CountVectorizer is: 
    0.9173
    
{'clf': VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('mnb', MultinomialNB()), ('svc', SVC())]), 'vect__max_df': 0.8, 'vect__max_features': 500, 'vect__ngram_range': (1, 2), 'vect__stop_words': 'english'}

Fitting 3 folds for each of 160 candidates, totalling 480 fits
Best score for Tf-IDF Vectorizer is: 
    0.9156
    
{'clf': VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('mnb', MultinomialNB()), ('svc', SVC())]), 'vect__max_df': 0.9, 'vect__max_features': 500, 'vect__ngram_range': (1, 2), 'vect__stop_words': 'english'}



In [18]:
results_df = pd.DataFrame(grid_search.cv_results_)

In [20]:
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__C,param_clf__penalty,param_vect__max_df,param_vect__max_features,param_vect__ngram_range,...,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,8.337792,0.097530,1.525834,0.012855,LogisticRegression(solver='liblinear'),0.5,l1,0.9,300,"(1, 3)",...,0.886454,0.886341,0.886748,0.000498,139,0.891878,0.884903,0.895916,0.890899,0.004549
1,6.195457,0.659931,0.798023,0.024647,LogisticRegression(solver='liblinear'),0.5,l1,0.9,300,"(1, 3)",...,0.904382,0.892323,0.896711,0.005443,90,0.901345,0.899851,0.901394,0.900863,0.000717
2,4.397391,0.182171,1.160251,0.136918,LogisticRegression(solver='liblinear'),0.5,l1,0.9,300,"(1, 2)",...,0.886454,0.888335,0.887413,0.000768,135,0.891878,0.885401,0.895916,0.891065,0.004331
3,3.556233,0.082449,0.717388,0.031691,LogisticRegression(solver='liblinear'),0.5,l1,0.9,300,"(1, 2)",...,0.904382,0.892323,0.897043,0.005260,86,0.901345,0.900349,0.901892,0.901196,0.000639
4,9.100743,0.091858,1.513699,0.134462,LogisticRegression(solver='liblinear'),0.5,l1,0.9,500,"(1, 3)",...,0.891434,0.885344,0.888408,0.002486,132,0.890882,0.892875,0.894422,0.892726,0.001449


# Modeling Metrics With Confusion Matrix

Accuracy

Misclassification Rate

Sensitivity/Recall

Specificity

Precision

# ROC Curve